In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

In [2]:
holidaysdf = pd.read_csv('holidays_events.csv')
oildf = pd.read_csv('oil.csv')
storedf = pd.read_csv('stores.csv')
traindf = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')
transactionsdf = pd.read_csv('transactions.csv')

In [3]:
# Clairfying 
# Merge Data Frame Test
df1 = pd.DataFrame({'a': ['foo', 'baz'], 'b': [1, 2]})
df2 = pd.DataFrame({'a': ['foo', 'baz'], 'c': [3, 4]})

In [4]:
df1

,a,b
0,foo,1
1,baz,2


In [5]:
df2

,a,c
0,foo,3
1,baz,4


In [6]:
df1.merge(df2, how='left', on='a') # on--Column will left & make vales of df2 based on it 

,a,b,c
0,foo,1,3
1,baz,2,4


In [13]:
# Merege all data into one data Frame File
mastertraindf = pd.merge(traindf, holidaysdf, on='date', how='left') # left of date & leave date & make values based on it
mastertraindf = pd.merge(mastertraindf, oildf, on='date', how='left')
mastertraindf = pd.merge(mastertraindf, storedf, on='store_nbr', how='left')
mastertraindf = pd.merge(mastertraindf, transactionsdf, on=['store_nbr', 'date'], how='left')



In [14]:
mastertraindf = mastertraindf.rename(columns={'type_x':'holidaytype', 'type_y':'storetype'})

In [15]:
mastertraindf['test']=0

In [16]:
mastertraindf

,id,date,store_nbr,family,sales,onpromotion,holidaytype,locale,locale_name,description,transferred,dcoilwtico,city,state,storetype,cluster,transactions,test
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13,NaN,0
1,1,2013-01-01,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13,NaN,0
2,2,2013-01-01,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13,NaN,0
3,3,2013-01-01,1,BEVERAGES,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13,NaN,0
4,4,2013-01-01,1,BOOKS,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0


In [17]:
mastertraindf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 18 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   date          object 
 2   store_nbr     int64  
 3   family        object 
 4   sales         float64
 5   onpromotion   int64  
 6   holidaytype   object 
 7   locale        object 
 8   locale_name   object 
 9   description   object 
 10  transferred   object 
 11  dcoilwtico    float64
 12  city          object 
 13  state         object 
 14  storetype     object 
 15  cluster       int64  
 16  transactions  float64
 17  test          int64  
dtypes: float64(3), int64(5), object(10)
memory usage: 442.8+ MB


In [18]:
mastertraindf.corr()

,id,store_nbr,sales,onpromotion,dcoilwtico,cluster,transactions,test
id,1.000000,3.013381e-04,0.086102,0.206032,-8.390363e-01,-4.369541e-05,-0.028106,NaN
store_nbr,0.000301,1.000000e+00,0.041273,0.007744,1.494354e-14,-5.928399e-02,0.189656,NaN
sales,0.086102,4.127337e-02,1.000000,0.428241,-7.939345e-02,3.853669e-02,0.214848,NaN
onpromotion,0.206032,7.744102e-03,0.428241,1.000000,-1.524600e-01,5.701707e-03,0.025809,NaN
dcoilwtico,-0.839036,1.494354e-14,-0.079393,-0.152460,1.000000e+00,-4.187239e-14,0.018664,NaN
cluster,-0.000044,-5.928399e-02,0.038537,0.005702,-4.187239e-14,1.000000e+00,0.196110,NaN
transactions,-0.028106,1.896558e-01,0.214848,0.025809,1.866365e-02,1.961100e-01,1.000000,NaN
test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From Correlation matrix it seems not much correlation exist in between featreus. So we dont have any dependant features

In [20]:
df = mastertraindf.dropna()

In [21]:
df

,id,date,store_nbr,family,sales,onpromotion,holidaytype,locale,locale_name,description,transferred,dcoilwtico,city,state,storetype,cluster,transactions,test
73062,73062,2013-02-11,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73063,73063,2013-02-11,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73064,73064,2013-02-11,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73065,73065,2013-02-11,1,BEVERAGES,172.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73066,73066,2013-02-11,1,BOOKS,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0


In [22]:
df = df.drop(columns=['id'],axis =1)

In [23]:
df

,date,store_nbr,family,sales,onpromotion,holidaytype,locale,locale_name,description,transferred,dcoilwtico,city,state,storetype,cluster,transactions,test
73062,2013-02-11,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73063,2013-02-11,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73064,2013-02-11,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73065,2013-02-11,1,BEVERAGES,172.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73066,2013-02-11,1,BOOKS,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054344,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054345,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054346,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0


In [24]:
# make date in appropriate type 
df['date'] = pd.to_datetime(df['date'])

In [25]:
df 

,date,store_nbr,family,sales,onpromotion,holidaytype,locale,locale_name,description,transferred,dcoilwtico,city,state,storetype,cluster,transactions,test
73062,2013-02-11,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73063,2013-02-11,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73064,2013-02-11,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73065,2013-02-11,1,BEVERAGES,172.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
73066,2013-02-11,1,BOOKS,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054344,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054345,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0
3054346,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0


In [28]:
def process_date(df):
    date_parts = ["year", "weekday", "month", 'day']
    for part in date_parts:
        
        df[part] = getattr(df['date'].dt, part).astype(int) # corresponding to pandas date time
    
    return df


In [29]:
df['date'] = process_date(df)

In [31]:
df = df.drop(columns=['date'],axis=1)

In [32]:
df

,store_nbr,family,sales,onpromotion,holidaytype,locale,locale_name,description,transferred,dcoilwtico,city,state,storetype,cluster,transactions,test,year,weekday,month,day
73062,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0,2013,0,2,11
73063,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0,2013,0,2,11
73064,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0,2013,0,2,11
73065,1,BEVERAGES,172.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0,2013,0,2,11
73066,1,BOOKS,0.000,0,Holiday,National,Ecuador,Carnaval,False,97.01,Quito,Pichincha,D,13,396.0,0,2013,0,2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0,2017,1,8,15
3054344,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0,2017,1,8,15
3054345,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0,2017,1,8,15
3054346,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2155.0,0,2017,1,8,15


In [44]:
df = df.drop(columns=['test'],axis=1)

In [45]:
categorical_col = ['family','holidaytype','locale','locale_name','description','transferred','storetype','cluster']

In [49]:
for col in categorical_col:
    df[col] = LabelEncoder().fit_transform(df[col])

In [50]:
df

,store_nbr,family,sales,onpromotion,holidaytype,locale,locale_name,description,transferred,dcoilwtico,city,state,storetype,cluster,transactions,year,weekday,month,day
73062,1,0,0.000,0,3,1,4,11,0,97.01,Quito,Pichincha,3,12,396.0,2013,0,2,11
73063,1,1,0.000,0,3,1,4,11,0,97.01,Quito,Pichincha,3,12,396.0,2013,0,2,11
73064,1,2,0.000,0,3,1,4,11,0,97.01,Quito,Pichincha,3,12,396.0,2013,0,2,11
73065,1,3,172.000,0,3,1,4,11,0,97.01,Quito,Pichincha,3,12,396.0,2013,0,2,11
73066,1,4,0.000,0,3,1,4,11,0,97.01,Quito,Pichincha,3,12,396.0,2013,0,2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,9,28,438.133,0,3,0,19,26,0,47.57,Quito,Pichincha,1,5,2155.0,2017,1,8,15
3054344,9,29,154.553,1,3,0,19,26,0,47.57,Quito,Pichincha,1,5,2155.0,2017,1,8,15
3054345,9,30,2419.729,148,3,0,19,26,0,47.57,Quito,Pichincha,1,5,2155.0,2017,1,8,15
3054346,9,31,121.000,8,3,0,19,26,0,47.57,Quito,Pichincha,1,5,2155.0,2017,1,8,15


In [52]:
df['city'] = LabelEncoder().fit_transform(df['city'])
df['state'] = LabelEncoder().fit_transform(df['state'])

In [53]:
df

,store_nbr,family,sales,onpromotion,holidaytype,locale,locale_name,description,transferred,dcoilwtico,city,state,storetype,cluster,transactions,year,weekday,month,day
73062,1,0,0.000,0,3,1,4,11,0,97.01,18,12,3,12,396.0,2013,0,2,11
73063,1,1,0.000,0,3,1,4,11,0,97.01,18,12,3,12,396.0,2013,0,2,11
73064,1,2,0.000,0,3,1,4,11,0,97.01,18,12,3,12,396.0,2013,0,2,11
73065,1,3,172.000,0,3,1,4,11,0,97.01,18,12,3,12,396.0,2013,0,2,11
73066,1,4,0.000,0,3,1,4,11,0,97.01,18,12,3,12,396.0,2013,0,2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,9,28,438.133,0,3,0,19,26,0,47.57,18,12,1,5,2155.0,2017,1,8,15
3054344,9,29,154.553,1,3,0,19,26,0,47.57,18,12,1,5,2155.0,2017,1,8,15
3054345,9,30,2419.729,148,3,0,19,26,0,47.57,18,12,1,5,2155.0,2017,1,8,15
3054346,9,31,121.000,8,3,0,19,26,0,47.57,18,12,1,5,2155.0,2017,1,8,15


In [54]:
numerical_col = ['onpromotion','dcoilwtico','transactions']

In [56]:
df[numerical_col] = StandardScaler().fit_transform(df[numerical_col])

In [57]:
df

,store_nbr,family,sales,onpromotion,holidaytype,locale,locale_name,description,transferred,dcoilwtico,city,state,storetype,cluster,transactions,year,weekday,month,day
73062,1,0,0.000,-0.232517,3,1,4,11,0,1.30955,18,12,3,12,-1.273056,2013,0,2,11
73063,1,1,0.000,-0.232517,3,1,4,11,0,1.30955,18,12,3,12,-1.273056,2013,0,2,11
73064,1,2,0.000,-0.232517,3,1,4,11,0,1.30955,18,12,3,12,-1.273056,2013,0,2,11
73065,1,3,172.000,-0.232517,3,1,4,11,0,1.30955,18,12,3,12,-1.273056,2013,0,2,11
73066,1,4,0.000,-0.232517,3,1,4,11,0,1.30955,18,12,3,12,-1.273056,2013,0,2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,9,28,438.133,-0.232517,3,0,19,26,0,-0.65644,18,12,1,5,0.399006,2017,1,8,15
3054344,9,29,154.553,-0.168378,3,0,19,26,0,-0.65644,18,12,1,5,0.399006,2017,1,8,15
3054345,9,30,2419.729,9.260063,3,0,19,26,0,-0.65644,18,12,1,5,0.399006,2017,1,8,15
3054346,9,31,121.000,0.280595,3,0,19,26,0,-0.65644,18,12,1,5,0.399006,2017,1,8,15


In [58]:
X = df.drop(columns=['sales'],axis=1)
y = df['sales']

In [59]:
X.shape

(300003, 18)

In [60]:
y.shape

(300003,)

In [61]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.2)

In [62]:
XGB = XGBRegressor().fit(x_train, y_train)

In [65]:
from sklearn.metrics import mean_squared_error
RMSE = [mean_squared_error(y_valid, XGB.predict(x_valid))**0.5 ]
R2_Score = [r2_score(y_valid, XGB.predict(x_valid)) ]
MAPE = [mean_absolute_percentage_error(y_valid, XGB.predict(x_valid)) ]

In [66]:
evaluation = pd.DataFrame({'RMSE':RMSE, 'R2_Score':R2_Score,'MAPE':MAPE})

In [67]:
evaluation

,RMSE,R2_Score,MAPE
0,536.841855,0.809512,3.016805e+16


In [69]:
XGB.score(x_valid,y_valid)

0.8095115619187061